# Домашнее задание №1: линейная регрессия (максимум 10 баллов)

Некоторые задания будут по вариантам (всего 4 варианта). Чтобы выяснить свой вариант, посчитайте количество букв в своей фамилии, возьмете остаток от деления на 4 и прибавьте 1.

In [1]:
import numpy as np

In [2]:
variant = len('Radnaev') % 4 + 1
print(variant)

4


## Многомерная линейная регрессия из sklearn

Создадим набор данных для многомерной регрессии

In [3]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples = 10000)
print(X.shape, y.shape)

(10000, 100) (10000,)


У нас 10000 объектов и 100 признаков. Для начала решим задачу аналитически "из коробки".

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X, y)
print(mean_squared_error(y, reg.predict(X)))

1.6307991658961891e-25


Теперь попробуем обучить линейную регрессию методом градиентного спуска "из коробки"

In [5]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(alpha=0.00000001).fit(X, y)
print(mean_squared_error(y, reg.predict(X)))
reg.coef_

2.7770664529808338e-12


array([-1.08714506e-09,  7.92405620e+01,  2.61857128e-08,  1.34422827e-08,
        4.10305900e-08, -3.11657198e-08,  6.15275078e-09, -1.68028578e-08,
       -1.49174276e-09,  3.04118450e-08,  6.26882182e-08,  1.09625896e-08,
        7.58596228e-08,  1.49992283e-08, -4.06921612e-09,  2.25734012e-08,
        1.93077055e-08,  4.55846649e-08,  2.37434067e-08,  9.85693079e-09,
        6.50897283e-08,  4.45478115e-08,  9.41942395e+01, -1.52533051e-08,
        3.41428433e-08,  7.15022805e-09, -3.79269384e-08, -9.83465591e-09,
       -3.87311643e-08, -7.40013651e-08, -3.87748354e-08,  5.93993545e-10,
       -1.37272381e-08,  2.63130888e-09, -3.14397194e-08, -6.04534018e-09,
        2.04614574e+01, -2.65953054e-08, -1.58126790e-08,  5.16605172e+01,
        5.28073427e-08,  2.05023140e-08,  7.99913803e-10, -1.56483942e-08,
       -2.41408432e-08, -5.23435808e-08, -4.16183908e-08, -3.25128623e-08,
        7.71380708e-08,  4.43503838e+01,  2.66237968e+01, -6.05424583e-08,
       -6.84239871e-08, -

***Задание 1 (1 балл).*** Объясните, чем вызвана разница в значениях двух полученных значений метрики?
1) Разница вызвана тем, что LinearRegression решает задачу аналитическим методом, а SGDRegressor является численным методом, в этом случае результат зависит от значений гиперпараметров.

***Задание 2 (1 балл).*** Подберите гиперпараметры в методе градиентного спуска так, чтобы значение MSE было близко к значению MSE, полученному при обучении LinearRegression.

In [6]:
reg = SGDRegressor(alpha=1e-6, max_iter=100000, tol=1e-4, penalty=None, learning_rate='constant', eta0=0.01)
#Уменьшили шаг, увеличили кол-во итераций, усилили критерий остановки.

reg.fit(X, y)

# Рассчитываем MSE
print("MSE для SGDRegressor:", mean_squared_error(y, reg.predict(X)))

MSE для SGDRegressor: 2.3186866294852795e-27


## Ваша многомерная линейная регрессия

***Задание 3 (5 баллов)***. Напишите собственную многомерную линейную регрессию, оптимизирующую MSE методом *градиентного спуска*. Для этого используйте шаблонный класс.

Критерий останова: либо норма разности весов на текущей и предыдущей итерациях меньше определенного значения (первый и третий варианты), либо модуль разности функционалов качества (MSE) на текущей и предыдущей итерациях меньше определенного значения (второй и четвертый варианты). Также предлагается завершать обучение в любом случае, если было произведено слишком много итераций.

***Задание 4 (2 балла)***. Добавьте l1 (первый и четвертый варианты) или l2 (второй и третий варианты) регуляризацию.

In [7]:

class LinearRegression(object):
    def __init__(self, alpha=0.1, l_ratio=0.00001, tol=1e-6, max_iter=10000):
        self.alpha = alpha
        self.l_ratio = l_ratio
        self.tol = tol
        self.max_iter = max_iter
        self.w = None  # Weights will be initialized during fit

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)

        mse_prev = float('inf')

        for iteration in range(self.max_iter):
            
            y_pred = X @ self.w
            
            error = y_pred - y
            
            mse = (1 / n_samples) * np.dot(error, error)

            
            if abs(mse_prev - mse) < self.tol:
                
                break
            mse_prev = mse

            
            reg = self.l_ratio * np.sign(self.w)
            reg[0] = 0 # убираем регуляризацию у свободных весов
            grad = (2 / n_samples) * (X.T @ error) + reg

            
            self.w -= self.alpha * grad

        else:
            print(f"Достигли макс. итераций: {self.max_iter}")

    def predict(self, X):
        return X @ self.w



In [8]:
my_reg = LinearRegression()
my_reg.fit(X, y)
assert mean_squared_error(y, my_reg.predict(X)) < 1e-3
mse = mean_squared_error(y, my_reg.predict(X))
print(f"Mean Squared Error: {mse}")
print('You are amazing! Great work!')

Mean Squared Error: 1.6824027281759088e-06
You are amazing! Great work!


***Задание 5 (1 балл)***. Обучите линейную регрессию из коробки с l1-регуляризацией (from sklearn.linear_model import Lasso, первый и четвертый варианты) или с l2-регуляризацией (from sklearn.linear_model import Ridge, второй и третий варианты) с значением параметра регуляризации 0.1. Обучите вашу линейную регрессию с тем же значением параметра регуляризации и сравните результаты. Сделайте выводы.

In [9]:
from sklearn.linear_model import Lasso
X, y = make_regression(n_samples=10000, n_features=100)
model = Lasso(alpha = 0.1)
model.fit(X, y)
mse = mean_squared_error(y, model.predict(X))
print(mse)

0.09998481668184668


Встроенные модели scikit-learn обычно более оптимизированы и эффективны для реальных задач. Они реализуют продвинутые методы оптимизации и обеспечивают лучшую производительность.